In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './국토교통부_건축물대장_층별개요/'
filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))&(f.startswith('floor'))]
filenames

['floor_info_baseline_seoul_201412.csv',
 'floor_info_baseline_seoul_201512.csv',
 'floor_info_baseline_seoul_201612.csv',
 'floor_info_baseline_seoul_201712.csv',
 'floor_info_baseline_seoul_201812.csv',
 'floor_info_baseline_seoul_201912.csv',
 'floor_info_baseline_seoul_202011.csv']

In [3]:
dfs_list = []
for file in filenames:
    df = pd.read_csv(basedir + file)
    dfs_list.append(df)

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
first_cols = dfs_list[0].columns

In [5]:
for df in dfs_list:
    print(first_cols == df.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


ValueError: Lengths must match to compare

In [6]:
for df in dfs_list:
    print(df.shape)

(956287, 40)
(2931131, 34)
(2945430, 34)
(2952425, 34)
(2963206, 34)
(2977877, 34)
(2986252, 34)


In [7]:
second_cols = dfs_list[1].columns
for i, df in enumerate(dfs_list):
    if i == 0:
        pass
    else:
        print(second_cols == df.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  Tr

In [8]:
first_cols

Index(['마트_층별_개요_PK', '건축물대장_PK', '대지_위치', '도로명_대지_위치', '건물_명', '시군구_코드',
       '법정동_코드', '대지_구분_코드', '번', '지', '특수지_명', '블럭', '로트', '지번_좌표_X',
       '지번_좌표_Y', '도로명_좌표_X', '도로명_좌표_Y', '기타_기재_사항', '새주소_도로_코드',
       '새주소_법정동_코드', '새주소_지상지하_코드', '새주소_본_번', '새주소_부_번', '동_명칭', '층_구분_코드',
       '층_구분_코드_명', '층_번호', '층_번호_명', '구조_코드', '구조_코드_명', '기타_구조', '주_용도_코드',
       '주_용도_코드_명', '기타_용도', '면적', '주_부속_구분_코드', '주_부속_구분_코드_명', '면적_제외_여부',
       '생성_일자', '기준년월'],
      dtype='object')

In [9]:
second_cols

Index(['관리_건축물대장_PK', '대지_위치', '도로명_대지_위치', '건물_명', '시군구_코드', '법정동_코드',
       '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '새주소_도로_코드', '새주소_법정동_코드',
       '새주소_지상지하_코드', '새주소_본_번', '새주소_부_번', '동_명', '층_구분_코드', '층_구분_코드_명',
       '층_번호', '층_번호_명', '구조_코드', '구조_코드_명', '기타_구조', '주_용도_코드', '주_용도_코드_명',
       '기타_용도', '면적(㎡)', '주_부속_구분_코드', '주_부속_구분_코드_명', '면적_제외_여부', '생성_일자',
       '기준년월'],
      dtype='object')

In [10]:
unshared_cols = []
for col in first_cols:
    if col not in second_cols:
        unshared_cols.append(col)

In [11]:
unshared_cols

['마트_층별_개요_PK',
 '건축물대장_PK',
 '블럭',
 '지번_좌표_X',
 '지번_좌표_Y',
 '도로명_좌표_X',
 '도로명_좌표_Y',
 '기타_기재_사항',
 '동_명칭',
 '면적']

In [12]:
dfs_list[0] = dfs_list[0].rename(columns={'건축물대장_PK':'관리_건축물대장_PK', '블럭':'블록', '동_명칭':'동_명', '면적':'면적(㎡)'})

In [13]:
dfs_list[0] = dfs_list[0].drop(columns=['마트_층별_개요_PK', '지번_좌표_X', '지번_좌표_Y', '도로명_좌표_X', '도로명_좌표_Y',
                                        '기타_기재_사항'
                                       ])

In [14]:
first_cols = dfs_list[0].columns

In [15]:
for df in dfs_list:
    print(first_cols == df.columns)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  Tr

In [16]:
concat_df = pd.concat(dfs_list)

In [17]:
concat_df.reset_index(drop=True, inplace=True)

In [18]:
print(concat_df.shape)
concat_df.head()

(18712608, 34)


,관리_건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,동_명,층_구분_코드,층_구분_코드_명,층_번호,층_번호_명,구조_코드,구조_코드_명,기타_구조,주_용도_코드,주_용도_코드_명,기타_용도,면적(㎡),주_부속_구분_코드,주_부속_구분_코드_명,면적_제외_여부,생성_일자,기준년월
0,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,11440,12600.0,0.0,81.0,29.0,NaN,NaN,NaN,1.144041e+11,12601.0,0.0,9.0,6.0,NaN,20.0,지상,1,1층,21.0,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,다세대주택,57.62,0.0,주건축물,NaN,20141109,201412
1,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,11440,12600.0,0.0,81.0,29.0,NaN,NaN,NaN,1.144041e+11,12601.0,0.0,9.0,6.0,NaN,20.0,지상,2,2층,21.0,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,다세대주택,67.20,0.0,주건축물,NaN,20141109,201412
2,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,11440,12600.0,0.0,81.0,29.0,NaN,NaN,NaN,1.144041e+11,12601.0,0.0,9.0,6.0,NaN,20.0,지상,3,3층,21.0,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,다세대주택,67.20,0.0,주건축물,NaN,20141109,201412
3,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,11440,12600.0,0.0,81.0,29.0,NaN,NaN,NaN,1.144041e+11,12601.0,0.0,9.0,6.0,NaN,20.0,지상,4,4층,21.0,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,다세대주택,59.68,0.0,주건축물,NaN,20141109,201412
4,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,11440,12600.0,0.0,81.0,29.0,NaN,NaN,NaN,1.144041e+11,12601.0,0.0,9.0,6.0,NaN,30.0,옥탑,1,옥탑 1층,21.0,철근콘크리트구조,철근콘크리트구조,02003,다세대주택,계단실(연면적제외),7.68,0.0,주건축물,NaN,20141109,201412


In [19]:
concat_df.drop(columns=[col for col in concat_df.columns if col.endswith('_코드')], inplace=True)
concat_df.shape

(18712608, 24)

In [20]:
concat_df.columns = [col.replace('_코드_명', '') for col in concat_df.columns]
concat_df.head()

,관리_건축물대장_PK,대지_위치,도로명_대지_위치,건물_명,번,지,특수지_명,블록,로트,새주소_본_번,새주소_부_번,동_명,층_구분,층_번호,층_번호_명,구조,기타_구조,주_용도,기타_용도,면적(㎡),주_부속_구분,면적_제외_여부,생성_일자,기준년월
0,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,지상,1,1층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,57.62,주건축물,NaN,20141109,201412
1,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,지상,2,2층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412
2,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,지상,3,3층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412
3,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,지상,4,4층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,59.68,주건축물,NaN,20141109,201412
4,11440-3279202,서울특별시 마포구 중동 81-29번지,NaN,중동빌라,81.0,29.0,NaN,NaN,NaN,9.0,6.0,NaN,옥탑,1,옥탑 1층,철근콘크리트구조,철근콘크리트구조,다세대주택,계단실(연면적제외),7.68,주건축물,NaN,20141109,201412


In [21]:
def correct_addr(x):
    # x is a string
    if (pd.isna(x) == True)|(x == '없음'):
        return x
    else:
        x = x.replace('번지', '')
        if x[0] == ' ':
            for i in range(len(x)):
                if x[i] != ' ':
                    x = x[i:]
                    break
        
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x

In [22]:
concat_df['지번주소'] = concat_df['대지_위치'].apply(correct_addr)

In [24]:
# 생각해보니 건축물에 관한 정보, 특히 사용승인일짜 같은 정보가 없으므로, 기존에 있는 living_area_per_building.csv를 쓰는 게 낫겠음.